In [1]:
import open_clip
from open_clip import create_model_and_transforms, trace_model, get_tokenizer, create_loss
from PIL import Image
import torch
import numpy as np

model, _, preprocess = open_clip.create_model_and_transforms('RN50', pretrained='openai')
tokenizer = open_clip.get_tokenizer('ViT-B-32')

image = preprocess(Image.open("./images/text_image_dog_example.jpg")).unsqueeze(0) # preprocess(Image.open("./data/UCM/imgs/1.tif")).unsqueeze(0)
text = tokenizer(["a photo of a diagrams", "a photo of a dog or cat", "a photo of a cat and feline", "a photo of a fields"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1).numpy()

print("Label probs:", np.round(text_probs,2))  # prints: [[1., 0., 0.]]

Label probs: [[0. 1. 0. 0.]]


In [2]:
text_features.shape, image_features.shape

(torch.Size([4, 1024]), torch.Size([1, 1024]))

In [ ]:
# Model names: 
# ['RN50', 'RN50-quickgelu', 'RN101', 'RN101-quickgelu', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B-32', 'ViT-B-32-quickgelu', 'ViT-B-16', 'ViT-L-14', 'ViT-L-14-336']